In [7]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [3]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prev_add = add!(model, AdditionNode())
    
    x_prior = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1   = add!(model, constvar(:add_1, 1.0))

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    
    add_x_and_noise = add!(model, AdditionNode())
    
    x = add!(model, simplerandomvar(:x))
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = add!(model, datavar(:y, Float64))
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(model, x_prev_add)
    activate!(model, add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getmarginal(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [5]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [37]:
@btime kalman($data)

  914.932 μs (14333 allocations: 639.34 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=14.071042598099691, σ=14.002800840280099)
 Normal{Float64}(μ=15.09464658849146, σ=9.950371902099892)
 Normal{Float64}(μ=15.802166658494807, σ=8.137884587711596)
 Normal{Float64}(μ=11.987173983617827, σ=7.053456158585984)
 Normal{Float64}(μ=10.970404828805968, σ=6.311944030978033)
 Normal{Float64}(μ=10.468979976770662, σ=5.763904177042351)
 Normal{Float64}(μ=11.219486933868867, σ=5.337605126836238)
 Normal{Float64}(μ=14.691935440044732, σ=4.993761694389224)
 Normal{Float64}(μ=14.807665709153707, σ=4.708816093480111)
 Normal{Float64}(μ=15.47896568077596, σ=4.467670516087703)
 Normal{Float64}(μ=14.182399601440263, σ=4.2601432284230505)
 Normal{Float64}(μ=13.97853782878819, σ=4.079085082240021)
 Normal{Float64}(μ=14.628001011390618, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.2569002185809, σ=0.582706653680032)
 Normal{Float64}(μ=590.2194468444178, σ=0.5822126418123303)
 Normal{Float64}(μ=591.1986862097391, σ=0.5817198842703781)
 

In [35]:
@time kalman(data)

  0.001500 seconds (14.33 k allocations: 639.344 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=14.071042598099691, σ=14.002800840280099)
 Normal{Float64}(μ=15.09464658849146, σ=9.950371902099892)
 Normal{Float64}(μ=15.802166658494807, σ=8.137884587711596)
 Normal{Float64}(μ=11.987173983617827, σ=7.053456158585984)
 Normal{Float64}(μ=10.970404828805968, σ=6.311944030978033)
 Normal{Float64}(μ=10.468979976770662, σ=5.763904177042351)
 Normal{Float64}(μ=11.219486933868867, σ=5.337605126836238)
 Normal{Float64}(μ=14.691935440044732, σ=4.993761694389224)
 Normal{Float64}(μ=14.807665709153707, σ=4.708816093480111)
 Normal{Float64}(μ=15.47896568077596, σ=4.467670516087703)
 Normal{Float64}(μ=14.182399601440263, σ=4.2601432284230505)
 Normal{Float64}(μ=13.97853782878819, σ=4.079085082240021)
 Normal{Float64}(μ=14.628001011390618, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.2569002185809, σ=0.582706653680032)
 Normal{Float64}(μ=590.2194468444178, σ=0.5822126418123303)
 Normal{Float64}(μ=591.1986862097391, σ=0.5817198842703781)
 

In [34]:
@time kalman(data)

  1.961765 seconds (4.79 M allocations: 255.983 MiB, 3.36% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=14.071042598099691, σ=14.002800840280099)
 Normal{Float64}(μ=15.09464658849146, σ=9.950371902099892)
 Normal{Float64}(μ=15.802166658494807, σ=8.137884587711596)
 Normal{Float64}(μ=11.987173983617827, σ=7.053456158585984)
 Normal{Float64}(μ=10.970404828805968, σ=6.311944030978033)
 Normal{Float64}(μ=10.468979976770662, σ=5.763904177042351)
 Normal{Float64}(μ=11.219486933868867, σ=5.337605126836238)
 Normal{Float64}(μ=14.691935440044732, σ=4.993761694389224)
 Normal{Float64}(μ=14.807665709153707, σ=4.708816093480111)
 Normal{Float64}(μ=15.47896568077596, σ=4.467670516087703)
 Normal{Float64}(μ=14.182399601440263, σ=4.2601432284230505)
 Normal{Float64}(μ=13.97853782878819, σ=4.079085082240021)
 Normal{Float64}(μ=14.628001011390618, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.2569002185809, σ=0.582706653680032)
 Normal{Float64}(μ=590.2194468444178, σ=0.5822126418123303)
 Normal{Float64}(μ=591.1986862097391, σ=0.5817198842703781)
 